This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import pandas as pd
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
# I_KEY = 'HXnXzRczj9Q3kzqrGoW3'
query = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02&api_key=" + API_KEY
r = requests.get(query)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.text)

{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2017-01-01","end_date":"2017-01-02","frequency":"daily","data":[["2017-01-02",34.99,35.94,34.99,35.8,null,44700.0,1590561.0,null,null,null]],"collapse":null,"order":null}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 

On inspection of the API documentation, we can see that the format of the time-series API is `https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}?api_key=YOURAPIKEY`. Using this information, we can substitute the database_code for the Frankfurt Stock Exchange (FSE), the dataset_code for Carl Zeiss (AFX_X), and our API key to acquire the data with the `requests` library.

In [5]:
query = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
r = requests.get(query)

#### 2. Convert the returned JSON object into a Python dictionary.

Once we have the data downloaded, we are able to use the `json` package to convert the text data to a dictionary. We isolate the column names and data and load it with `pd.DataFrame` which will accept a list of lists as the body of a `DataFrame`. We can then convert the date information to datetime format and set it as the dataframe index for use while isolating specific timeframes from the data. We inspect the output to confirm that it is the format we were attempting to achieve.

In [6]:
# Convert data
json_data = json.loads(r.text)
col_names = json_data['dataset_data']['column_names']
afx_data = json_data['dataset_data']['data']
data = pd.DataFrame(afx_data, columns=col_names)

# Convert dates to datetime and set as dataframe index for slicing
data.Date = pd.to_datetime(data.Date)
data.set_index(data.Date, inplace=True)
data = data['2017']

# Confirm data is correctly formatted
print(data.index)
print(data.head())

DatetimeIndex(['2017-12-29', '2017-12-28', '2017-12-27', '2017-12-22',
               '2017-12-21', '2017-12-20', '2017-12-19', '2017-12-18',
               '2017-12-15', '2017-12-14',
               ...
               '2017-01-13', '2017-01-12', '2017-01-11', '2017-01-10',
               '2017-01-09', '2017-01-06', '2017-01-05', '2017-01-04',
               '2017-01-03', '2017-01-02'],
              dtype='datetime64[ns]', name='Date', length=255, freq=None)
                 Date   Open   High    Low  Close  Change  Traded Volume  \
Date                                                                       
2017-12-29 2017-12-29  51.76  51.94  51.45  51.76     NaN        34640.0   
2017-12-28 2017-12-28  51.65  51.82  51.43  51.60     NaN        40660.0   
2017-12-27 2017-12-27  51.45  51.89  50.76  51.82     NaN        57452.0   
2017-12-22 2017-12-22  51.05  51.50  50.92  51.32     NaN        71165.0   
2017-12-21 2017-12-21  51.16  51.52  50.90  51.40     NaN       120649.0   

   

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

Finding the maximum and minimum values of the Open column is as easy as calling a `pandas Series` method.

In [7]:
max_2017 = data.Open.max()
min_2017 = data.Open.min()
print('The maximum opening price of AFX_X was %.2f.' % max_2017)
print('The minimum opening price was %.2f.' % min_2017)

The maximum opening price of AFX_X was 53.11.
The minimum opening price was 34.00.


#### 4. What was the largest change in any one day (based on High and Low price)?

We can calculate this value from the absolute value and max methods chained from our dataframe. We have to do a bit of extra work to distinguish between negative and positive changes. We can do this by finding the index position of the absolute value max change and isolate that value.

In [8]:
data['day_change'] = (data.High - data.Low)
max_change_index = data.day_change.abs().idxmax()
max_change_val = data.loc[max_change_index].day_change
print("The largest change in one day was %.2f on %s." % (max_change_val, max_change_index.strftime('%b %d, %Y')))

The largest change in one day was 2.81 on May 11, 2017.


#### 5. What was the largest change between any two days (based on Closing Price)?

We can create a dataframe with all indices shifted one place and calculate the change based on closing price from two successive days. We can again chain `abs` and `idxmax` methods to get the position of the largest change.

In [9]:
data['lagged'] = data.shift(1).Close - data.Close
lagged_max_index = data.lagged.abs().idxmax()
lagged_max_val = data.loc[lagged_max_index].lagged
print('The largest day-to-day change in value was %.2f on %s.' % (lagged_max_val, lagged_max_index.strftime('%b %d, %Y')))

The largest day-to-day change in value was -2.56 on Aug 08, 2017.


#### 6. What was the average daily trading volume during this year?

We can get average trading volume by using the `mean` method.

In [10]:
avg_trade_vol = data['Traded Volume'].mean()
print('The mean average trading volume for 2017 was %.2f.' % avg_trade_vol)

The mean average trading volume for 2017 was 89124.34.


#### 7. (Optional) What was the median trading volume during this year. 
#### (Note: you may need to implement your own function for calculating the median.)

The pandas series object has a `median` method for calculating the median of a series.

In [11]:
med_trade_vol = data['Traded Volume'].median()
print('The median trade volume for 2017 was %.2f.' % med_trade_vol)

The median trade volume for 2017 was 76286.00.
